# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [4]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

<select class="movie_name" id="sbox_mid" name="movie_id">
<option value="">請選擇片名</option>
<option data-name="白頭山：半島浩劫" value="10354">白頭山：半島浩劫</option>
<option data-name="CATS貓" value="10166">CATS貓</option>
<option data-name="海上花" value="10376">海上花</option>
<option data-name="奪魂露營車" value="10366">奪魂露營車</option>
<option data-name="世紀球王 馬拉度納" value="10361">世紀球王 馬拉度納</option>
<option data-name="怨靈古堡" value="10356">怨靈古堡</option>
<option data-name="添好孕" value="10332">添好孕</option>
<option data-name="外出偷馬" value="10310">外出偷馬</option>
<option data-name="茱蒂" value="10112">茱蒂</option>
<option data-name="燃燒女子的畫像" value="10069">燃燒女子的畫像</option>
<option data-name="葉問4：完結篇" value="9890">葉問4：完結篇</option>
<option data-name="STAR WARS : 天行者的崛起" value="10225">STAR WARS : 天行者的崛起</option>
<option data-name="復仇母親" value="10392">復仇母親</option>
<option data-name="風暴過後" value="10387">風暴過後</option>
<option data-name="南方車站的聚會" value="10380">南方車站的聚會</option>
<option data-name="祭旗" value="10373">祭旗</option>
<option

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [9]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10354

In [10]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': '.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozmoviesilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14
放映地區: 花蓮, 代號(area_id): 12
放映地區: 金門, 代號(area_id): 24


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [11]:
# 指定放映地區
area_id = 28

In [12]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 25
十二月 26
十二月 27
十二月 28
十二月 29


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [14]:
# 選定要觀看的日期
date = "2019-12-25"

In [16]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [48]:
print('----------------------------------------------------------------------')
for i in html:    
    print('電影院：{}'.format(i['data-theater_name']))
    movie_type = i.find('span', attrs={'class':'tapR'})
    print('放映類型：{}'.format(movie_type.text))
    time_table = i.find_all('input', attrs={'class':'gabtn'})
    for t in time_table:
        print(t['value'])
    print('----------------------------------------------------------------------')



----------------------------------------------------------------------
電影院：國賓影城(台北長春廣場)
放映類型：數位
2019-12-25 10:50:00
2019-12-25 13:10:00
2019-12-25 15:30:00
2019-12-25 16:50:00
2019-12-25 17:50:00
2019-12-25 19:10:00
2019-12-25 20:10:00
2019-12-25 21:30:00
2019-12-25 22:30:00
----------------------------------------------------------------------
電影院：欣欣秀泰影城
放映類型：數位
2019-12-25 11:30:00
2019-12-25 14:00:00
2019-12-25 16:30:00
2019-12-25 19:00:00
2019-12-25 21:30:00
2019-12-26 00:00:00
----------------------------------------------------------------------
電影院：台北美麗華大直影城
放映類型：數位
2019-12-25 13:25:00
2019-12-25 14:30:00
2019-12-25 17:00:00
2019-12-25 18:20:00
2019-12-25 19:30:00
2019-12-25 20:50:00
2019-12-25 22:00:00
2019-12-25 22:50:00
2019-12-26 00:30:00
----------------------------------------------------------------------
電影院：華威天母影城
放映類型：數位
2019-12-25 10:30:00
2019-12-25 13:00:00
2019-12-25 15:30:00
2019-12-25 18:00:00
2019-12-25 20:30:00
2019-12-25 23:00:00
-------------------------------